In [1]:
!conda activate nauman_gpu

**Requiremements**




In [ ]:
# conda install pytorch=1.12.0 torchvision=0.13.0 cudatoolkit=11.3.1 -c pytorch
# conda install albumentations=1.2.1 -c conda-forge
# conda install einops=0.4.1 -c conda-forge
# conda install torchsummary=1.5.1 -c ravelbio
# conda install tqdm=4.64.0
# conda install scikit-image=0.18.3
# conda install scikit-learn=1.1.1
# conda install torchmetrics=0.9.3 -c conda-forge

# **Data Downloading**
For training and evaluation we used the BraTS Dataset from RSNA-ASNR-MICCAI BraTS Continuous Evaluation Challenge:
https://www.synapse.org/#!Synapse:syn51156910/wiki/622351

1. Download the data
2. Convert 3d .nii.gz file to 2d
    2a. We used two modalities. i) T1 ii) T1 Contrast
3. Resize the images to 256 256, (Original images in 3D are 240 240 155)
4. We have placed the covnerted 2d resized files here:

https://drive.google.com/file/d/1hXJ8CP6BgJz2bFM1OF2LmGtc3VvRgZhS/view?usp=sharing


# **Preprocessing**


In [ ]:
import os
import sys
import numpy as np
import glob
import matplotlib.pyplot as plt
import nibabel as nib
import itertools
from PIL import Image

In [ ]:
# Converting T1 3D Nifti images to 2D PNG images 

folder = '/local/data0/home/naumanb/Codes/4_SwinUnetR_BraTS_2023/BraTS2023/TrainingData'

for root, dir, files in os.walk(folder):
    for file in files:
        im = os.path.join(root, file)
        # print(file)

        # if file.endswith('t1c.nii.gz'):
        if file.endswith('t1n.nii.gz'):
            print(file)
            img = os.path.join(root, file)
            img = nib.load(img)
            img = img.get_data().astype(np.float32)
            img_data = np.rot90(img, 3)

            plt.imshow(img_data[:, :, 77], cmap='gray')
            plt.axis('off')
            # Save as PNG
            plt.savefig('/local/data0/home/naumanb/Codes/8_Cx_GAN/data/t1n/' +
                        file+'.png', bbox_inches='tight', pad_inches=0)

            # Show the plot (optional)
            # plt.show()

    # break

In [ ]:
# Converting T1-Contrat 3D Nifti images to 2D PNG images

import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import nibabel as nib

folder = '/local/data0/home/naumanb/Codes/4_SwinUnetR_BraTS_2023/BraTS2023/TrainingData'

for root, dir, files in os.walk(folder):
    for file in files:
        im = os.path.join(root, file)

        if file.endswith('t1c.nii.gz'):
            print(file)
            img_path = os.path.join(root, file)
            img = nib.load(img_path)
            img_data = img.get_data().astype(np.float32)
            img_data = np.rot90(img_data, 3)


            plt.imshow(img_data[:, :, 77], cmap='gray')
            plt.axis('off')

            # Save as PNG
            output_path = '/local/data0/home/naumanb/Codes/8_Cx_GAN/data/t1c/' + file + '.png'
            plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
            # plt.show()

In [ ]:
# Resizing the images to 256x256

#Resize the image to 256,256
from PIL import Image
import os, sys

path = "/local/data0/home/naumanb/Codes/8_Cx_GAN/data/t1c/"
dirs = os.listdir( path )
out = "/local/data0/home/naumanb/Codes/8_Cx_GAN/data/t1c_r/"

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            print(item)
            print(f)
            imResize = im.resize((256,256), Image.ANTIALIAS)
            imResize.save(out+item+'.png', 'PNG', quality=100)

resize()

# **Data Loader**

In [2]:
import numpy as np
import tensorflow as tf
import tf2lib as tl


def make_dataset(img_paths, batch_size, load_size, crop_size, training, drop_remainder=True, shuffle=False, repeat=1):
    if training:
        @tf.function
        def _map_fn(img):  # preprocessing
            img = tf.image.random_flip_left_right(img)
            img = tf.image.resize(img, [load_size, load_size])
            img = tf.image.random_crop(img, [crop_size, crop_size, tf.shape(img)[-1]])
            img = tf.clip_by_value(img, 0, 255) / 255.0  # or img = tl.minmax_norm(img)
            img = img * 2 - 1
            return img
    else:
        @tf.function
        def _map_fn(img):  # preprocessing
            img = tf.image.resize(img, [crop_size, crop_size])  # or img = tf.image.resize(img, [load_size, load_size]); img = tl.center_crop(img, crop_size)
            img = tf.clip_by_value(img, 0, 255) / 255.0  # or img = tl.minmax_norm(img)
            img = img * 2 - 1
            return img

    return tl.disk_image_batch_dataset(img_paths,
                                       batch_size,
                                       drop_remainder=drop_remainder,
                                       map_fn=_map_fn,
                                       shuffle=shuffle,
                                       repeat=repeat)


def make_zip_dataset(A_img_paths, B_img_paths, batch_size, load_size, crop_size, training, shuffle=False, repeat=False):
    # zip two datasets aligned by the longer one
    if repeat:
        A_repeat = B_repeat = None  # cycle both
    else:
        if len(A_img_paths) >= len(B_img_paths):
            A_repeat = 1
            B_repeat = None  # cycle the shorter one
        else:
            A_repeat = None  # cycle the shorter one
            B_repeat = 1

    A_dataset = make_dataset(A_img_paths, batch_size, load_size, crop_size, training, drop_remainder=True, shuffle=shuffle, repeat=A_repeat)
    B_dataset = make_dataset(B_img_paths, batch_size, load_size, crop_size, training, drop_remainder=True, shuffle=shuffle, repeat=B_repeat)

    A_B_dataset = tf.data.Dataset.zip((A_dataset, B_dataset))
    len_dataset = max(len(A_img_paths), len(B_img_paths)) // batch_size

    return A_B_dataset, len_dataset


class ItemPool:

    def __init__(self, pool_size=50):
        self.pool_size = pool_size
        self.items = []

    def __call__(self, in_items):
        # `in_items` should be a batch tensor

        if self.pool_size == 0:
            return in_items

        out_items = []
        for in_item in in_items:
            if len(self.items) < self.pool_size:
                self.items.append(in_item)
                out_items.append(in_item)
            else:
                if np.random.rand() > 0.5:
                    idx = np.random.randint(0, len(self.items))
                    out_item, self.items[idx] = self.items[idx], in_item
                    out_items.append(out_item)
                else:
                    out_items.append(in_item)
        return tf.stack(out_items, axis=0)


2023-12-18 13:15:42.945429: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/local/data0/software/miniconda3/envs/nauman_gpu/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
2023-12-18 13:15:44.866949: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devi

# **Network**

In [3]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"Selected GPU: {logical_gpus[0].name}")
    except RuntimeError as e:
        print(e)


Selected GPU: /device:GPU:0


2023-12-18 13:15:46.748934: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 13:15:46.750792: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-18 13:15:46.751360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:68:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2023-12-18 13:15:46.751404: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-12-18 13:15:46.751434: I tensorflow/stream_executor/

In [4]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras


# ==============================================================================
# =                                  networks                                  =
# ==============================================================================

def _get_norm_layer(norm):
    if norm == 'none':
        return lambda: lambda x: x
    elif norm == 'batch_norm':
        return keras.layers.BatchNormalization
    elif norm == 'instance_norm':
        return tfa.layers.InstanceNormalization
    elif norm == 'layer_norm':
        return keras.layers.LayerNormalization


def ResnetGenerator(input_shape=(256, 256, 3),
                    output_channels=3,
                    dim=64,
                    n_downsamplings=2,
                    n_blocks=9,
                    norm='instance_norm'):
    Norm = _get_norm_layer(norm)

    def _residual_block(x):
        dim = x.shape[-1]
        h = x

        h = tf.pad(h, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='REFLECT')
        h = keras.layers.Conv2D(dim, 3, padding='valid', use_bias=False)(h)
        h = Norm()(h)
        h = tf.nn.relu(h)

        h = tf.pad(h, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='REFLECT')
        h = keras.layers.Conv2D(dim, 3, padding='valid', use_bias=False)(h)
        h = Norm()(h)

        return keras.layers.add([x, h])

    # 0
    h = inputs = keras.Input(shape=input_shape)

    # 1
    h = tf.pad(h, [[0, 0], [3, 3], [3, 3], [0, 0]], mode='REFLECT')
    h = keras.layers.Conv2D(dim, 7, padding='valid', use_bias=False)(h)
    h = Norm()(h)
    h = tf.nn.relu(h)

    # 2
    for _ in range(n_downsamplings):
        dim *= 2
        h = keras.layers.Conv2D(dim, 3, strides=2, padding='same', use_bias=False)(h)
        h = Norm()(h)
        h = tf.nn.relu(h)

    # 3
    for _ in range(n_blocks):
        h = _residual_block(h)

    # 4
    for _ in range(n_downsamplings):
        dim //= 2
        h = keras.layers.Conv2DTranspose(dim, 3, strides=2, padding='same', use_bias=False)(h)
        h = Norm()(h)
        h = tf.nn.relu(h)

    # 5
    h = tf.pad(h, [[0, 0], [3, 3], [3, 3], [0, 0]], mode='REFLECT')
    h = keras.layers.Conv2D(output_channels, 7, padding='valid')(h)
    h = tf.tanh(h)

    return keras.Model(inputs=inputs, outputs=h)


def ConvDiscriminator(input_shape=(256, 256, 3),
                      dim=64,
                      n_downsamplings=3,
                      norm='instance_norm'):
    dim_ = dim
    Norm = _get_norm_layer(norm)

    # 0
    h = inputs = keras.Input(shape=input_shape)

    # 1
    h = keras.layers.Conv2D(dim, 4, strides=2, padding='same')(h)
    h = tf.nn.leaky_relu(h, alpha=0.2)

    for _ in range(n_downsamplings - 1):
        dim = min(dim * 2, dim_ * 8)
        h = keras.layers.Conv2D(dim, 4, strides=2, padding='same', use_bias=False)(h)
        h = Norm()(h)
        h = tf.nn.leaky_relu(h, alpha=0.2)

    # 2
    dim = min(dim * 2, dim_ * 8)
    h = keras.layers.Conv2D(dim, 4, strides=1, padding='same', use_bias=False)(h)
    h = Norm()(h)
    h = tf.nn.leaky_relu(h, alpha=0.2)

    # 3
    h = keras.layers.Conv2D(1, 4, strides=1, padding='same')(h)

    return keras.Model(inputs=inputs, outputs=h)


# ==============================================================================
# =                          learning rate scheduler                           =
# ==============================================================================

class LinearDecay(keras.optimizers.schedules.LearningRateSchedule):
    # if `step` < `step_decay`: use fixed learning rate
    # else: linearly decay the learning rate to zero

    def __init__(self, initial_learning_rate, total_steps, step_decay):
        super(LinearDecay, self).__init__()
        self._initial_learning_rate = initial_learning_rate
        self._steps = total_steps
        self._step_decay = step_decay
        self.current_learning_rate = tf.Variable(initial_value=initial_learning_rate, trainable=False, dtype=tf.float32)

    def __call__(self, step):
        self.current_learning_rate.assign(tf.cond(
            step >= self._step_decay,
            true_fn=lambda: self._initial_learning_rate * (1 - 1 / (self._steps - self._step_decay) * (step - self._step_decay)),
            false_fn=lambda: self._initial_learning_rate
        ))
        return self.current_learning_rate


# **Training**

In [5]:
import functools
import imlib as im
import numpy as np
import pylib as py
import tensorflow as tf
import tensorflow.keras as keras
import tf2lib as tl
import tf2gan as gan
import tqdm
import glob
import os
from PIL import Image
import tensorflow as tf
import data
# import module
import warnings

# Disable all warnings
warnings.filterwarnings('ignore')

import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"


# ==============================================================================
# =                                   param                                    =
# ==============================================================================

dataset = 'brats'
datasets_dir = 'datasets'
load_size = 256
crop_size = 256
batch_size = 1
epochs = 100
epoch_decay = 50
lr = 0.0002
beta_1 = 0.5
adversarial_loss_mode = 'lsgan'
gradient_penalty_mode = 'none'
gradient_penalty_weight = 10.0
cycle_loss_weight = 10.0
identity_loss_weight = 0.0
pool_size = 50



# output_dir
output_dir = py.join('output_brats', dataset)
py.mkdir(output_dir)

# save settings
# save settings
settings = {
    'dataset': dataset,
    'datasets_dir': datasets_dir,
    'load_size': load_size,
    'crop_size': crop_size,
    'batch_size': batch_size,
    'epochs': epochs,
    'epoch_decay': epoch_decay,
    'lr': lr,
    'beta_1': beta_1,
    'adversarial_loss_mode': adversarial_loss_mode,
    'gradient_penalty_mode': gradient_penalty_mode,
    'gradient_penalty_weight': gradient_penalty_weight,
    'cycle_loss_weight': cycle_loss_weight,
    'identity_loss_weight': identity_loss_weight,
    'pool_size': pool_size,
}

py.args_to_yaml(py.join(output_dir, 'settings.yml'), type('', (), settings)())



# ==============================================================================
# =                                    data                                    =
# ==============================================================================


#-------------------------------------------- old Unpaired dataset --------------------------------------------#
A_img_paths = py.glob(py.join('datasets/train_folder', 't1n_r'), '*.png')
B_img_paths = py.glob(py.join('datasets/train_folder', 't1c_r'), '*.png')
A_B_dataset, len_dataset = data.make_zip_dataset(A_img_paths, B_img_paths, batch_size, load_size, crop_size, training=True, repeat=False)




A2B_pool = data.ItemPool(pool_size)
B2A_pool = data.ItemPool(pool_size)

A_img_paths_test = py.glob(py.join('datasets/test_folder/', 't1n_r'), '*.png')
B_img_paths_test = py.glob(py.join('datasets/test_folder/', 't1c_r'), '*.png')
A_B_dataset_test, _ = data.make_zip_dataset(A_img_paths_test, B_img_paths_test, batch_size, load_size, crop_size, training=False, repeat=True)


# ==============================================================================
# =                                   models                                   =
# ==============================================================================

G_A2B = ResnetGenerator(input_shape=(crop_size, crop_size, 3))
G_B2A = ResnetGenerator(input_shape=(crop_size, crop_size, 3))

D_A = ConvDiscriminator(input_shape=(crop_size, crop_size, 3))
D_B = ConvDiscriminator(input_shape=(crop_size, crop_size, 3))

d_loss_fn, g_loss_fn = gan.get_adversarial_losses_fn(adversarial_loss_mode)
cycle_loss_fn = tf.losses.MeanAbsoluteError()
identity_loss_fn = tf.losses.MeanAbsoluteError()

G_lr_scheduler = LinearDecay(lr, epochs * len_dataset, epoch_decay * len_dataset)
D_lr_scheduler = LinearDecay(lr, epochs * len_dataset, epoch_decay * len_dataset)
G_optimizer = keras.optimizers.Adam(learning_rate=G_lr_scheduler, beta_1=beta_1)
D_optimizer = keras.optimizers.Adam(learning_rate=D_lr_scheduler, beta_1=beta_1)


# ==============================================================================
# =                                 train step                                 =
# ==============================================================================

@tf.function
def train_G(A, B):
    with tf.GradientTape() as t:
        A2B = G_A2B(A, training=True)
        B2A = G_B2A(B, training=True)
        A2B2A = G_B2A(A2B, training=True)
        B2A2B = G_A2B(B2A, training=True)
        A2A = G_B2A(A, training=True)
        B2B = G_A2B(B, training=True)

        A2B_d_logits = D_B(A2B, training=True)
        B2A_d_logits = D_A(B2A, training=True)

        A2B_g_loss = g_loss_fn(A2B_d_logits)
        B2A_g_loss = g_loss_fn(B2A_d_logits)
        A2B2A_cycle_loss = cycle_loss_fn(A, A2B2A)
        B2A2B_cycle_loss = cycle_loss_fn(B, B2A2B)
        A2A_id_loss = identity_loss_fn(A, A2A)
        B2B_id_loss = identity_loss_fn(B, B2B)

        G_loss = (A2B_g_loss + B2A_g_loss) + (A2B2A_cycle_loss + B2A2B_cycle_loss) * cycle_loss_weight + (A2A_id_loss + B2B_id_loss) * identity_loss_weight

    G_grad = t.gradient(G_loss, G_A2B.trainable_variables + G_B2A.trainable_variables)
    G_optimizer.apply_gradients(zip(G_grad, G_A2B.trainable_variables + G_B2A.trainable_variables))

    return A2B, B2A, {'A2B_g_loss': A2B_g_loss,
                      'B2A_g_loss': B2A_g_loss,
                      'A2B2A_cycle_loss': A2B2A_cycle_loss,
                      'B2A2B_cycle_loss': B2A2B_cycle_loss,
                      'A2A_id_loss': A2A_id_loss,
                      'B2B_id_loss': B2B_id_loss}


@tf.function
def train_D(A, B, A2B, B2A):
    with tf.GradientTape() as t:
        A_d_logits = D_A(A, training=True)
        B2A_d_logits = D_A(B2A, training=True)
        B_d_logits = D_B(B, training=True)
        A2B_d_logits = D_B(A2B, training=True)

        A_d_loss, B2A_d_loss = d_loss_fn(A_d_logits, B2A_d_logits)
        B_d_loss, A2B_d_loss = d_loss_fn(B_d_logits, A2B_d_logits)
        D_A_gp = gan.gradient_penalty(functools.partial(D_A, training=True), A, B2A, mode=gradient_penalty_mode)
        D_B_gp = gan.gradient_penalty(functools.partial(D_B, training=True), B, A2B, mode=gradient_penalty_mode)

        D_loss = (A_d_loss + B2A_d_loss) + (B_d_loss + A2B_d_loss) + (D_A_gp + D_B_gp) * gradient_penalty_weight

    D_grad = t.gradient(D_loss, D_A.trainable_variables + D_B.trainable_variables)
    D_optimizer.apply_gradients(zip(D_grad, D_A.trainable_variables + D_B.trainable_variables))

    return {'A_d_loss': A_d_loss + B2A_d_loss,
            'B_d_loss': B_d_loss + A2B_d_loss,
            'D_A_gp': D_A_gp,
            'D_B_gp': D_B_gp}


def train_step(A, B):
    A2B, B2A, G_loss_dict = train_G(A, B)

    # cannot autograph `A2B_pool`
    A2B = A2B_pool(A2B)  # or A2B = A2B_pool(A2B.numpy()), but it is much slower
    B2A = B2A_pool(B2A)  # because of the communication between CPU and GPU

    D_loss_dict = train_D(A, B, A2B, B2A)

    return G_loss_dict, D_loss_dict


@tf.function
def sample(A, B):
    A2B = G_A2B(A, training=False)
    B2A = G_B2A(B, training=False)
    A2B2A = G_B2A(A2B, training=False)
    B2A2B = G_A2B(B2A, training=False)
    return A2B, B2A, A2B2A, B2A2B


# ==============================================================================
# =                                    run                                     =
# ==============================================================================

# epoch counter
ep_cnt = tf.Variable(initial_value=0, trainable=False, dtype=tf.int64)

# checkpoint
checkpoint = tl.Checkpoint(dict(G_A2B=G_A2B,
                                G_B2A=G_B2A,
                                D_A=D_A,
                                D_B=D_B,
                                G_optimizer=G_optimizer,
                                D_optimizer=D_optimizer,
                                ep_cnt=ep_cnt),
                           py.join(output_dir, 'checkpoints'),
                           max_to_keep=5)
try:  # restore checkpoint including the epoch counter
    checkpoint.restore().assert_existing_objects_matched()
except Exception as e:
    print(e)

# summary
train_summary_writer = tf.summary.create_file_writer(py.join(output_dir, 'summaries', 'train'))

# sample
test_iter = iter(A_B_dataset_test)
sample_dir = py.join(output_dir, 'samples_training')
py.mkdir(sample_dir)

# main loop
with train_summary_writer.as_default():
    for ep in tqdm.trange(epochs, desc='Epoch Loop'):
        if ep < ep_cnt:
            continue

        # update epoch counter
        ep_cnt.assign_add(1)

        # train for an epoch
        for A, B in tqdm.tqdm(A_B_dataset, desc='Inner Epoch Loop', total=len_dataset):
            G_loss_dict, D_loss_dict = train_step(A, B)

            # # summary
            tl.summary(G_loss_dict, step=G_optimizer.iterations, name='G_losses')
            tl.summary(D_loss_dict, step=G_optimizer.iterations, name='D_losses')
            tl.summary({'learning rate': G_lr_scheduler.current_learning_rate}, step=G_optimizer.iterations, name='learning rate')

            # sample
            if G_optimizer.iterations.numpy() % 100 == 0:
                A, B = next(test_iter)
                A2B, B2A, A2B2A, B2A2B = sample(A, B)
                img = im.immerge(np.concatenate([A, A2B, A2B2A, B, B2A, B2A2B], axis=0), n_rows=2)
                im.imwrite(img, py.join(sample_dir, 'iter-%09d.jpg' % G_optimizer.iterations.numpy()))

        # save checkpoint
        checkpoint.save(ep)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

2023-12-18 13:16:38.484353: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-12-18 13:16:38.496116: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3300000000 Hz


No checkpoint specified (save_path=None); nothing is being restored.


Epoch Loop:   0%|          | 0/100 [00:00<?, ?it/s]2023-12-18 13:16:53.172102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-12-18 13:16:54.700088: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Epoch Loop: 100%|██████████| 100/100 [7:34:25<00:00, 272.65s/it]


# **Test**

In [ ]:
import imlib as im
import numpy as np
import pylib as py
import tensorflow as tf
import tf2lib as tl

import data
import module
import numpy as np
import imageio as iio
import os.path as py
# Replace these lines
import pylib as py
import os.path as py

# With something like this
import pylib as py



# ==============================================================================
# =                                   param                                    =
# ==============================================================================

# py.arg('--experiment_dir')
# py.arg('--batch_size', type=int, default=32)
# test_args = py.args()
# args = py.args_from_yaml(py.join(test_args.experiment_dir, 'settings.yml'))
# args.__dict__.update(test_args.__dict__)


load_size= 286
crop_size= 256
batch_size= 1


# ==============================================================================
# =                                    test                                    =
# ==============================================================================

# data
A_img_paths_test = py.glob(py.join('datasets/test_folder', 't1n_r'), '*.png')
B_img_paths_test = py.glob(py.join('datasets/test_folder', 't1c_r'), '*.png')

A_dataset_test = data.make_dataset(A_img_paths_test,batch_size,load_size, crop_size,
                                   training=False, drop_remainder=False, shuffle=False, repeat=1)

B_dataset_test = data.make_dataset(B_img_paths_test, batch_size, load_size, crop_size,
                                   training=False, drop_remainder=False, shuffle=False, repeat=1)

# model
G_A2B = module.ResnetGenerator(input_shape=(crop_size, crop_size, 3))
G_B2A = module.ResnetGenerator(input_shape=(crop_size, crop_size, 3))

# resotre
tl.Checkpoint(dict(G_A2B=G_A2B, G_B2A=G_B2A), py.join('output_brats/brats/', 'checkpoints')).restore()


@tf.function
def sample_A2B(A):
    A2B = G_A2B(A, training=False)
    A2B2A = G_B2A(A2B, training=False)
    return A2B, A2B2A


@tf.function
def sample_B2A(B):
    B2A = G_B2A(B, training=False)
    B2A2B = G_A2B(B2A, training=False)
    return B2A, B2A2B


# run
save_dir = py.join('output_brats/brats/', 'samples_testing', 'A2B')
py.mkdir(save_dir)
i = 0
for A in A_dataset_test:
    A2B, A2B2A = sample_A2B(A)
    for A_i, A2B_i, A2B2A_i in zip(A, A2B, A2B2A):
        img = np.concatenate([A_i.numpy(), A2B_i.numpy(), A2B2A_i.numpy()], axis=1)
        # im.imwrite(img, py.join(save_dir, py.name_ext(A_img_paths_test[i])))
        iio.imsave(py.join(save_dir, py.name_ext(A_img_paths_test[i])), img)
        i += 1

save_dir = py.join('output_brats/brats/', 'samples_testing', 'B2A')
py.mkdir(save_dir)
i = 0
for B in B_dataset_test:
    B2A, B2A2B = sample_B2A(B)
    for B_i, B2A_i, B2A2B_i in zip(B, B2A, B2A2B):
        img = np.concatenate([B_i.numpy(), B2A_i.numpy(), B2A2B_i.numpy()], axis=1)
        # im.imwrite(img, py.join(save_dir, py.name_ext(B_img_paths_test[i])))
        iio.imsave(py.join(save_dir, py.name_ext(B_img_paths_test[i])), img)
        i += 1

In [7]:
from skimage.metrics import mean_squared_error, structural_similarity, peak_signal_noise_ratio

# Function to preprocess images (if needed)
def preprocess_image(image):
    # You can add preprocessing steps here if necessary
    return image.numpy()

# Placeholder arrays for storing metrics
mse_a2b_values = []
ssim_a2b_values = []
psnr_a2b_values = []

mse_b2a_values = []
ssim_b2a_values = []
psnr_b2a_values = []

# Calculate metrics for A2B
for A in A_dataset_test:
    A2B, _ = sample_A2B(A)
    for A_i, A2B_i in zip(A, A2B):
        A_i = preprocess_image(A_i)
        A2B_i = preprocess_image(A2B_i)

        mse_value = mean_squared_error(A_i, A2B_i)
        ssim_value = structural_similarity(A_i, A2B_i, multichannel=True)
        psnr_value = tf.image.psnr(A_i, A2B_i, max_val=1.0)

        mse_a2b_values.append(mse_value)
        ssim_a2b_values.append(ssim_value)
        psnr_a2b_values.append(psnr_value)

# Calculate metrics for B2A
for B in B_dataset_test:
    _, B2A = sample_B2A(B)
    for B_i, B2A_i in zip(B, B2A):
        B_i = preprocess_image(B_i)
        B2A_i = preprocess_image(B2A_i)

        mse_value = mean_squared_error(B_i, B2A_i)
        ssim_value = structural_similarity(B_i, B2A_i, multichannel=True)
        psnr_value = tf.image.psnr(B_i, B2A_i, max_val=1.0)

        mse_b2a_values.append(mse_value)
        ssim_b2a_values.append(ssim_value)
        psnr_b2a_values.append(psnr_value)

# Calculate mean values
mean_mse_a2b = np.mean(mse_a2b_values)
mean_ssim_a2b = np.mean(ssim_a2b_values)
mean_psnr_a2b = np.mean(psnr_a2b_values)

mean_mse_b2a = np.mean(mse_b2a_values)
mean_ssim_b2a = np.mean(ssim_b2a_values)
mean_psnr_b2a = np.mean(psnr_b2a_values)

# Print results
print("A2B Metrics:")
print("Mean Squared Error:", mean_mse_a2b)
print("SSIM:", mean_ssim_a2b)
print("PSNR:", mean_psnr_a2b)

print("\nB2A Metrics:")
print("Mean Squared Error:", mean_mse_b2a)
print("SSIM:", mean_ssim_b2a)
print("PSNR:", mean_psnr_b2a)


A2B Metrics:
Mean Squared Error: 0.11810231861677417
SSIM: 0.6296439
PSNR: 9.880537

B2A Metrics:
Mean Squared Error: 0.006030355766295807
SSIM: 0.88735026
PSNR: 23.636662
